In [1]:
# import create engine from sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

In [5]:
# load csv members table 

members_df = pd.read_csv('/Users/ruhel/Desktop/Members.csv',index_col=False)
bookings_df = pd.read_csv('/Users/ruhel/Desktop/Bookings.csv',index_col=False)
facilities_df = pd.read_csv('/Users/ruhel/Desktop/Facilities.csv',index_col=False)
print(members_df.columns.tolist())
print(bookings_df.columns.tolist())
print(facilities_df.columns.tolist())
# create engine 

engine = create_engine('sqlite:///country_club.sqlite')

# save the dataframes as sql table

members_df.to_sql('members', con=engine, if_exists='replace',index=False)
bookings_df.to_sql('bookings',con=engine, if_exists='replace',index=False)
facilities_df.to_sql('facilities',con=engine, if_exists='replace',index=False)

['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']
['bookid', 'facid', 'memid', 'starttime', 'slots']
['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']


9

In [6]:
# Quering the Members table 

df_members=pd.read_sql('SELECT * FROM members', con=engine)

In [7]:
df_members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,NaN,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,NaN,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1.0,2012-07-03 10:25:05


In [8]:
# Quering the bookings table
df_booking=pd.read_sql('SELECT * FROM bookings', con=engine)
df_booking.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [9]:
# Quering the facilities table
df_facilities=pd.read_sql('SELECT * FROM facilities', con=engine)
df_facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**


In [17]:
sql_query = """
               SELECT f.name AS facility_name, 
                        SUM(CASE WHEN b.memid=0 THEN f.guestcost*b.slots
                                 ELSE b.memid * b.slots END) AS Total_revenue
               FROM bookings AS b
               INNER JOIN facilities AS f
               ON f.facid = b.facid
               GROUP BY f.name
               Having Total_revenue < 1000
               ORDER BY Total_revenue DESC;
               """
Total_revenue = pd.read_sql(sql_query, con=engine)
print(Total_revenue)

Empty DataFrame
Columns: [facility_name, Total_revenue]
Index: []


**NO facility meets the condition.**

**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**


In [21]:
query="""
          SELECT m1.surname AS member_surname,
          m1.firstname AS member_firstname,
          m2.surname AS recommenders_surname,
          m2.firstname AS recommenders_firstname
          FROM members AS m1
          LEFT JOIN members AS m2
          ON m1.memid=m2.recommendedby
          ORDER BY m1.surname, m1.firstname;
          """
members_report=pd.read_sql(query, con=engine) 
print(members_report.head())

  member_surname member_firstname recommenders_surname recommenders_firstname
0          Bader         Florence               Sarwin              Ramnaresh
1          Baker             Anne                 None                   None
2          Baker          Timothy               Coplin                   Joan
3         Boothe              Tim                 None                   None
4        Butters           Gerald              Genting                Matthew


**Q12: Find the facilities with their usage by member, but not guests**


In [23]:
Query1="""
          SELECT f.name AS facility,
                 SUM(b.slots) AS Total_bookings
           FROM bookings AS b
           LEFT JOIN facilities AS f
           ON f.facid=b.facid
           WHERE b.memid != 0
           GROUP BY f.name
           ORDER BY Total_bookings DESC;
        """
Usage_by_member=pd.read_sql(Query1, con=engine)
print(Usage_by_member.head())

          facility  Total_bookings
0  Badminton Court            1086
1   Tennis Court 1             957
2   Massage Room 1             884
3   Tennis Court 2             882
4    Snooker Table             860


**Q13: Find the facilities usage by month, but not guests**

In [30]:
Query2 = """ 
            SELECT f.name AS facility_name,
            strftime ('%m', b.starttime) AS month,
            SUM(b.slots) AS Total_bookings
            FROM facilities AS f
            JOIN bookings AS b
            ON b.facid=f.facid
            WHERE b.memid !=0
            GROUP BY f.name
            ORDER BY month;

            """
usage_by_month = pd.read_sql(Query2, con=engine)
print(usage_by_month.head())

     facility_name month  Total_bookings
0  Badminton Court    07            1086
1   Massage Room 1    07             884
2   Massage Room 2    07              54
3       Pool Table    07             856
4    Snooker Table    07             860
